In [3]:
import pandas as pd
from splinter import Browser
import time
import pymongo

In [4]:
#Set up browser
url = 'https://www.espn.com/nfl/stats/player/_/stat/receiving'
executable_path = {'executable_path': 'C:/Users/Daniel/Desktop/Drivers/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#loop through page to scrape data at the end
browser.visit(url)
flag=True
while flag:
    try:
        browser.click_link_by_text('Show More')
        time.sleep(1)
    except:
        print('cycle complete')
        flag= False
if flag==False:
    receiving_tables= pd.read_html(browser.html)
    
#join tables   
df=receiving_tables[0]
df_2=receiviing_tables[1] 
#join the tables
mf=df.join(df_2)

#clean the team names from the player names
mf['Name'] = mf['Name'].astype(str)
mf[['Team']]= mf['Name'].str.extract('([A-Z][A-Z]+)',expand =True)
mf['Name'] = mf.apply(lambda x: x['Name'].replace(x['Team'],""), axis=1)
#add into mongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db=client.NFL_StatsDB
collection=db.receiving
data1 = mf.to_dict(orient='records')
collection.insert_many(data1)

cycle complete


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 17 columns):
RK       431 non-null int64
Name     431 non-null object
POS      431 non-null object
GP       431 non-null int64
REC      431 non-null int64
TGTS     431 non-null int64
YDS      431 non-null int64
AVG      431 non-null float64
TD       431 non-null int64
LNG      431 non-null int64
BIG      431 non-null int64
YDS/G    431 non-null float64
FUM      431 non-null int64
LST      431 non-null int64
YAC      431 non-null int64
FD       431 non-null int64
Team     431 non-null object
dtypes: float64(2), int64(12), object(3)
memory usage: 57.4+ KB


In [20]:
# # clean team names from player names in the table
# teams_list=[]
# names_list=[]
# for item in df.Name:
#     if item[-3].isupper():
#         teams_list.append(item[-3:])
#         names_list.append(item[:-3])
#     else:
#         teams_list.append(item[-2:])
#         names_list.append(item[:-2])
# df.Name=names_list
# df['Team Name']=teams_list

##look into camelcase to help with stripping issue
##look into regex library

NameError: name 'df' is not defined